# Simulação de Jogador Automático de Forca

### Passo 1: Importar Bibliotecas e Configurações Iniciais
Primeiro, vamos importar as bibliotecas necessárias e definir as classes e funções básicas.

In [3]:
import requests
import random
import math
from collections import Counter
from tqdm import tqdm  # Para acompanhamento do progresso


### Passo 2: Definir a Classe `JogadorAutomatico`

A classe `JogadorAutomatico` é responsável por carregar o vocabulário e calcular a entropia de cada letra para escolher as melhores opções ao jogar.

In [4]:
class JogadorAutomatico:
    def __init__(self):
        self.vocabulario = self.carregar_vocabulario()
        self.possibilidades = []

    def carregar_vocabulario(self):
        # Baixa o vocabulário do link fornecido
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        response = requests.get(url)
        if response.status_code == 200:
            vocabulario = response.text.splitlines()
            return vocabulario
        else:
            print(f"Erro ao carregar vocabulário: {response.status_code}")
            return []

    def iniciar_jogo(self, tamanho_palavra):
        # Filtra o vocabulário para palavras do mesmo tamanho
        self.possibilidades = [palavra for palavra in self.vocabulario if len(palavra) == tamanho_palavra]
        self.letras_usadas = set()  # Reset para cada novo jogo

    def calcular_entropia_letra_rapido(self):
        # Calcula a entropia somente para letras que ainda não foram usadas
        letras_disponiveis = Counter("".join(self.possibilidades))
        total_letras = sum(letras_disponiveis.values())
        entropia = {
            letra: -(freq / total_letras) * math.log2(freq / total_letras)
            for letra, freq in letras_disponiveis.items() if letra not in self.letras_usadas
        }
        return entropia

    def escolher_letra_rapido(self):
        # Seleciona a letra com maior entropia, evitando letras já tentadas
        entropia = self.calcular_entropia_letra_rapido()
        letra_escolhida = max(entropia, key=entropia.get)
        self.letras_usadas.add(letra_escolhida)  # Marca a letra como usada
        return letra_escolhida

#### Explicação:

1. **carregar_vocabulario**: Carrega a lista de palavras a partir de uma URL.
2. **iniciar_jogo**: Filtra as palavras para manter apenas aquelas com o mesmo tamanho da palavra que será adivinhada.
3. **calcular_entropia_letra_rapido**: Calcula a entropia das letras restantes, considerando apenas as letras que ainda não foram usadas.
4. **escolher_letra_rapido**: Escolhe a letra com maior entropia (ou seja, a letra mais informativa para o jogo).

### Passo 3: Definir a Classe `JogoDeForca`

A classe `JogoDeForca` contém a lógica do jogo da forca, como escolher a palavra secreta, verificar tentativas de letras e determinar o término do jogo.

In [5]:
class JogoDeForca:
    def __init__(self):
        self.vocabulario = JogadorAutomatico().carregar_vocabulario()

    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.vocabulario)
        self.palavra_oculta = ["_"] * len(self.palavra)  # Representa a palavra como "_" para cada letra
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                indices = [idx for idx, l in enumerate(self.palavra) if l == letra]
                for i in indices:
                    self.palavra_oculta[i] = letra
                return indices
            else:
                self.vidas -= 1
                return []
        else:
            return False

    def tentar_palavra(self, palavra):
        if palavra == self.palavra:
            return True
        else:
            self.vidas = 0
            return False

    def jogo_terminado(self):
        return "_" not in self.palavra_oculta or self.vidas == 0

#### Explicação:

1. **novo_jogo**: Inicia o jogo com uma palavra aleatória e define o número de vidas.
2. **tentar_letra**: Verifica se a letra está presente na palavra e atualiza as vidas.
3. **tentar_palavra**: Permite uma tentativa de adivinhar a palavra completa.
4. **jogo_terminado**: Retorna `True` se o jogo terminou (vitória ou perda de todas as vidas).

### Passo 4: Integrar `JogadorAutomatico` com o `JogoDeForca`

Agora, a classe `JogadorAutomaticoIntegrado` une o jogador automático ao jogo, permitindo que o jogador jogue o jogo completo com a lógica da entropia.

In [6]:
class JogadorAutomaticoIntegrado(JogadorAutomatico):
    def jogar(self, jogo):
        # Inicia um novo jogo e define as possibilidades com base no tamanho da palavra
        tamanho_palavra = jogo.novo_jogo()
        self.iniciar_jogo(tamanho_palavra)

        while not jogo.jogo_terminado():
            letra = self.escolher_letra_rapido()
            resultado = jogo.tentar_letra(letra)
            
            # Atualiza as possibilidades com base na resposta do jogo
            if resultado:
                self.possibilidades = [p for p in self.possibilidades if all(p[i] == letra for i in resultado)]
            else:
                self.possibilidades = [p for p in self.possibilidades if letra not in p]

            # Tentativa de adivinhar a palavra se restar uma única possibilidade
            if len(self.possibilidades) == 1:
                palavra_final = self.possibilidades[0]
                return jogo.tentar_palavra(palavra_final)
        return jogo.jogo_terminado()

#### Explicação:

- **jogar**: Método principal onde o jogador automático escolhe letras e tenta adivinhar a palavra, atualizando as possibilidades conforme o jogo avança.

### Passo 5: Simular Vários Jogos e Avaliar Desempenho

Por fim, executamos uma simulação de múltiplos jogos e calculamos o desempenho do jogador automático.

In [7]:
def simular_jogos_rapido(n=100):
    vitorias = 0
    derrotas = 0
    tentativas_totais = []
    palavras_dificeis = []
    
    for _ in tqdm(range(n), desc="Simulando Jogos"):
        jogo = JogoDeForca()
        jogador = JogadorAutomaticoIntegrado()
        
        # Inicia o jogo e realiza as jogadas
        resultado = jogador.jogar(jogo)
        
        # Contabiliza o resultado
        if resultado:
            vitorias += 1
            tentativas_totais.append(5 - jogo.vidas)
        else:
            derrotas += 1
            palavras_dificeis.append(jogo.palavra)

    # Calcula as taxas de sucesso e derrota
    taxa_sucesso = vitorias / n * 100
    taxa_derrota = derrotas / n * 100
    media_tentativas = sum(tentativas_totais) / len(tentativas_totais) if tentativas_totais else 0

    # Exibe os resultados
    print(f"\nSimulação de {n} jogos:")
    print(f"Vitórias: {vitorias}")
    print(f"Derrotas: {derrotas}")
    print(f"Taxa de Sucesso: {taxa_sucesso:.2f}%")
    print(f"Taxa de Derrota: {taxa_derrota:.2f}%")
    print(f"Média de Tentativas em Vitórias: {media_tentativas:.2f}")

    # Análise de palavras difíceis
    print("\nAnálise de Palavras Difíceis:")
    if palavras_dificeis:
        contagem_palavras_dificeis = Counter(palavras_dificeis)
        palavras_mais_dificeis = contagem_palavras_dificeis.most_common(5)
        for palavra, freq in palavras_mais_dificeis:
            print(f"Palavra: {palavra}, Ocorrências: {freq}")
    else:
        print("Nenhuma palavra específica causou dificuldade consistentemente.")

# Executa a simulação
simular_jogos_rapido(100)

Simulando Jogos: 100%|██████████| 100/100 [03:42<00:00,  2.22s/it]


Simulação de 100 jogos:
Vitórias: 100
Derrotas: 0
Taxa de Sucesso: 100.00%
Taxa de Derrota: 0.00%
Média de Tentativas em Vitórias: 1.44

Análise de Palavras Difíceis:
Nenhuma palavra específica causou dificuldade consistentemente.


#### Explicação:

- **simular_jogos_rapido**: Roda uma simulação de `n` jogos (100 por padrão) e mostra a taxa de sucesso, taxa de derrota e palavras mais difíceis.